In [ ]:
!pip install  xgboost==1.7.6 polars snowflake-connector-python --quiet

In [ ]:
import polars as pl
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import warnings
import pickle
import snowflake.connector
warnings.filterwarnings('ignore')

In [ ]:
connection_parameters = {
    "account": "xxx.us-west-2",
    "user": "xxx",
    "password": "xxx",
    "role": "ACCOUNTADMIN",
    "warehouse": "flight_wh",
    "database": "flight_pricing",
    "schema": "ANALYTICS"
}

print("Connecting to Snowflake...")
ctx = snowflake.connector.connect(**connection_parameters)
cs = ctx.cursor()

query = "SELECT * FROM ML_FLIGHT_PRICING SAMPLE(30)"
cs.execute(query)

print("Fetching data")
arrow_table = cs.fetch_arrow_all()

df = pl.from_arrow(arrow_table)

print(f"Loaded {df.height:,} rows into Polars.")
print(f"Memory usage: {df.estimated_size() / 1e9:.2f} GB")
print(df.head())

cs.close()
ctx.close()

Connecting to Snowflake...
Fetching data as Arrow...
Loaded 24,647,324 rows into Polars.
Memory usage: 1.48 GB
shape: (5, 19)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ TOTAL_FAR ┆ FARE_BASI ┆ DAYS_UNTI ┆ SEATS_REM ┆ … ┆ IS_HOLIDA ┆ IS_BASIC_ ┆ IS_NON_ST ┆ IS_REFUN │
│ E         ┆ S_CODE    ┆ L_FLIGHT  ┆ AINING    ┆   ┆ Y         ┆ ECONOMY   ┆ OP        ┆ DABLE    │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ str       ┆ i8        ┆ i8        ┆   ┆ i8        ┆ i8        ┆ i8        ┆ i8       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 138.6     ┆ Q7AHZNB3  ┆ 12        ┆ 7         ┆ … ┆ 0         ┆ 1         ┆ 1         ┆ 0        │
│ 201.6     ┆ SVAIZNN3  ┆ 45        ┆ 3         ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ 0        │
│ 477.61    ┆ HAA0OHEN  ┆ 48        ┆ 9         ┆ … ┆ 0         ┆ 

In [20]:
df=df.drop("FLIGHT_DATE_KEY")

In [21]:
# 1. Class Mappings
first_class = ['F', 'A']
business_class = ['J', 'C', 'D', 'I', 'Z', 'P']
premium_eco = ['W', 'S', 'T', 'R']
full_economy = ['Y', 'B', 'M', 'H']
discount_economy = ['K', 'L', 'Q', 'V', 'U', 'X', 'N', 'O', 'G', 'E']

print("Parsing Fare Basis Codes with Safe Heuristics...")

# 2. Initial Parsing
df = df.with_columns([
    pl.col('FARE_BASIS_CODE').fill_null('UNKNOWN').alias('FARE_BASIS_CODE')
])

df = df.with_columns([
    pl.col('FARE_BASIS_CODE').str.slice(0, 1).str.to_uppercase().alias('booking_class_letter'),
    pl.col('FARE_BASIS_CODE').str.slice(1).alias('fare_suffix')
])

# 3. Apply Logic
df = df.with_columns([
    # Cabin Category Mapping
    pl.when(pl.col('booking_class_letter').is_in(first_class)).then(pl.lit('First'))
      .when(pl.col('booking_class_letter').is_in(business_class)).then(pl.lit('Business'))
      .when(pl.col('booking_class_letter').is_in(premium_eco)).then(pl.lit('Premium Economy'))
      .when(pl.col('booking_class_letter').is_in(full_economy)).then(pl.lit('Full Economy'))
      .when(pl.col('booking_class_letter').is_in(discount_economy)).then(pl.lit('Discount Economy'))
      .otherwise(pl.lit('Unknown'))
      .alias('cabin_category'),

    # Numeric Rule Extraction
    pl.col('FARE_BASIS_CODE').str.extract(r'(\d+)', 1)
      .cast(pl.Int32, strict=False)
      .fill_null(0)
      .alias('fare_rule_number'),

    # Passenger Type
    pl.when(pl.col('fare_suffix').str.to_uppercase().str.contains(r'(CH|CNN)'))
      .then(pl.lit('Child'))
      .when(pl.col('fare_suffix').str.to_uppercase().str.contains(r'(IN|INF)'))
      .then(pl.lit('Infant'))
      .otherwise(pl.lit('Adult'))
      .alias('passenger_type'),

    # Seasonality 
    pl.when(pl.col('FARE_BASIS_CODE').str.to_uppercase().str.ends_with('H'))
      .then(pl.lit('High'))
      .when(pl.col('FARE_BASIS_CODE').str.to_uppercase().str.ends_with('L'))
      .then(pl.lit('Low'))
      .otherwise(pl.lit('Standard'))
      .alias('seasonality_proxy')
])

# 4. Feature Flags
df = df.with_columns([
    pl.when(pl.col('fare_rule_number') > 0).then(1).otherwise(0).alias('has_numeric_rule'),
    pl.col('fare_suffix').str.contains('N').cast(pl.Int8).alias('is_night_fare_proxy'),
    pl.col('fare_suffix').str.contains('W').cast(pl.Int8).alias('is_weekend_fare_proxy')
])

# Cleanup
df = df.drop(['booking_class_letter', 'fare_suffix'])

print("Fare parsing complete.")

Parsing Fare Basis Codes with Safe Heuristics...
Fare parsing complete.


In [22]:
print(f"Before filtering: {len(df)} rows")

zero_seats_count = len(df.filter(pl.col("SEATS_REMAINING") == 0))
print(f"Rows with 0 seats: {zero_seats_count} ({zero_seats_count/len(df)*100:.2f}%)")

df = df.filter(pl.col("SEATS_REMAINING") != 0)

print(f"After filtering: {len(df)} rows")

Before filtering: 24647324 rows
Rows with 0 seats: 1603969 (6.51%)
After filtering: 23043355 rows


In [23]:
print(f"Rows before cleaning: {len(df)}")

df = df.filter(
    (pl.col('TOTAL_FARE') > 0) &
    (pl.col('TOTAL_TRAVEL_DISTANCE') > 0) &
    (pl.col('TRAVEL_DURATION_MINUTES') > 20) &
    (pl.col('SEATS_REMAINING') >= 0) &
    (pl.col('DAYS_UNTIL_FLIGHT') >= 0)
)

print(f"Rows after removing negatives/zeros: {len(df)}")



def get_bounds(df, col_name):
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3 - q1
    
    upper_bound = q3 + (2.5 * iqr)
    lower_bound = q1 - (2.5 * iqr)
    
    return lower_bound, upper_bound

low_price, high_price = get_bounds(df, 'TOTAL_FARE')
print(f"Price Limits: {low_price} to {high_price}")

df = df.filter(pl.col('TOTAL_FARE') <= high_price)


low_dur, high_dur = get_bounds(df, 'TRAVEL_DURATION_MINUTES')
print(f"Duration Limits: {low_dur} to {high_dur}")

df = df.filter(pl.col('TRAVEL_DURATION_MINUTES') <= high_dur)


print(f"Final clean rows count: {len(df)}")

Rows before cleaning: 23043355
Rows after removing negatives/zeros: 22292306
Price Limits: -445.15 to 1117.85
Duration Limits: -485.0 to 1291.0
Final clean rows count: 22154170


In [24]:
df = df.drop("FARE_BASIS_CODE")

In [26]:
df_pandas = df.to_pandas()
str_cols = [
    'AIRLINE_CODE', 
    'ORIGIN_CITY', 
    'DEST_CITY',
    'cabin_category',
    'passenger_type',
    'seasonality_proxy'
]

encoders = {}

for col in str_cols:
    print(f"   -> Encoding {col}...")
    le = LabelEncoder()
    df_pandas[col] = le.fit_transform(df_pandas[col].astype(str))
    encoders[col] = le
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)
print(df_pandas.dtypes)

   -> Encoding AIRLINE_CODE...
   -> Encoding ORIGIN_CITY...
   -> Encoding DEST_CITY...
   -> Encoding cabin_category...
   -> Encoding passenger_type...
   -> Encoding seasonality_proxy...
TOTAL_FARE                 float64
DAYS_UNTIL_FLIGHT             int8
SEATS_REMAINING               int8
TOTAL_TRAVEL_DISTANCE      float64
TRAVEL_DURATION_MINUTES      int16
NUM_SEGMENTS                  int8
AIRLINE_CODE                 int64
ORIGIN_CITY                  int64
DEST_CITY                    int64
FLIGHT_YEAR                  int16
FLIGHT_MONTH                  int8
FLIGHT_DAY_OF_WEEK            int8
IS_WEEKEND                    int8
IS_HOLIDAY                    int8
IS_BASIC_ECONOMY              int8
IS_NON_STOP                   int8
IS_REFUNDABLE                 int8
cabin_category               int64
fare_rule_number             int32
passenger_type               int64
seasonality_proxy            int64
has_numeric_rule             int32
is_night_fare_proxy           int8
is_w

In [28]:
X = df_pandas.drop(columns=['TOTAL_FARE'])
y = df_pandas['TOTAL_FARE']

print("Splitting data")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Splitting data


In [31]:
model = xgb.XGBRegressor(
    n_estimators=1500,
    learning_rate=0.1,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42,
    early_stopping_rounds=50
)

model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=10)
p1 = model.predict(X_test)
print(f"   -> XGB Score: {r2_score(y_test, p1)*100:.2f}%")

[0]	validation_0-rmse:351.19397
[10]	validation_0-rmse:149.93491
[20]	validation_0-rmse:94.30547
[30]	validation_0-rmse:81.80603
[40]	validation_0-rmse:77.73394
[50]	validation_0-rmse:75.34143
[60]	validation_0-rmse:73.93645
[70]	validation_0-rmse:72.43086
[80]	validation_0-rmse:71.21862
[90]	validation_0-rmse:70.09828
[100]	validation_0-rmse:68.95800
[110]	validation_0-rmse:68.21265
[120]	validation_0-rmse:67.41177
[130]	validation_0-rmse:66.75128
[140]	validation_0-rmse:66.36265
[150]	validation_0-rmse:65.89156
[160]	validation_0-rmse:65.49131
[170]	validation_0-rmse:65.01960
[180]	validation_0-rmse:64.50524
[190]	validation_0-rmse:64.09973
[200]	validation_0-rmse:63.77320
[210]	validation_0-rmse:63.43798
[220]	validation_0-rmse:63.16887
[230]	validation_0-rmse:62.96403
[240]	validation_0-rmse:62.71763
[250]	validation_0-rmse:62.48717
[260]	validation_0-rmse:62.27210
[270]	validation_0-rmse:62.05993
[280]	validation_0-rmse:61.84218
[290]	validation_0-rmse:61.58124
[300]	validation_0-

In [33]:
model.save_model("final_flight_pricing_model.json")

In [ ]:
print("\nEvaluating Model...")
predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print(f"--------------------------------")
print(f"R2 Score (Accuracy): {r2*100:.2f}%")
print(f"MAE (Average Error): ${mae:.2f}")
print(f"RMSE: ${rmse:.2f}")
print(f"--------------------------------")


Evaluating Model...
--------------------------------
R2 Score (Accuracy): 91.24%
MAE (Average Error): $33.97
RMSE: $51.93
--------------------------------
Model saved successfully as JSON!


In [37]:
print("\nFeature Importance:")
importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(importance.head(10))


Feature Importance:
                  Feature  Importance
13       IS_BASIC_ECONOMY    0.433447
4            NUM_SEGMENTS    0.095170
18         passenger_type    0.075580
14            IS_NON_STOP    0.066589
2   TOTAL_TRAVEL_DISTANCE    0.050745
16         cabin_category    0.044078
5            AIRLINE_CODE    0.035144
20       has_numeric_rule    0.034332
17       fare_rule_number    0.032573
22  is_weekend_fare_proxy    0.030607
